<a href="https://colab.research.google.com/github/marco-siino/fake_news_spreaders_detection/blob/main/FNS_XLNet_MSiino_ModelNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset. 
XLNet Model, Training and Testing Notebook.
Code by M. Siino. 

From the paper: "Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset." by M.Siino et al.

## Importing modules.

In [1]:
!pip install simpletransformers
!pip install tensorboardx

import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import torch
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from google.colab import files
from io import open
from pathlib import Path
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs


     |████████████████████████████████| 248 kB 11.9 MB/s 
     |████████████████████████████████| 1.2 MB 32.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 3.8 MB 41.9 MB/s 
     |████████████████████████████████| 325 kB 49.3 MB/s 
     |████████████████████████████████| 1.7 MB 37.3 MB/s 
     |████████████████████████████████| 9.9 MB 22.2 MB/s 
     |████████████████████████████████| 6.5 MB 37.4 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 895 kB 25.3 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
     |████████████████████████████████| 144 kB 43.3 MB/s 
     |████████████████████████████████| 181 kB 17.2 MB/s 
     |████████████████████████████████| 63 kB 1.0 MB/s 
     |████████████████████████████████| 212 kB 36.8 MB/s 
     |████████████████████████████████| 1.1 MB 32.7 MB/s 
     |████████████████████████████████| 134 kB 13.2 MB/s 
     |██████████████

     |████████████████████████████████| 125 kB 11.5 MB/s 


## Importing DS and extract in current working directory.

In [2]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-training-2020-02-23.zip"
urlTestSet="https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-test-2020-02-23.zip"

training_set = tf.keras.utils.get_file("pan20-author-profiling-training-2020-02-23.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan20-author-profiling-test-2020-02-23.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan20-author-profiling-training-2020-02-23')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan20-author-profiling-test-2020-02-23')

print(training_set)
print(training_set_dir)

!ls -A

2146304/2135236 [==============================] - 0s 0us/step
./pan20-author-profiling-training-2020-02-23.zip
./pan20-author-profiling-training-2020-02-23
.config
__MACOSX
pan20-author-profiling-test-2020-02-23
pan20-author-profiling-test-2020-02-23.zip
pan20-author-profiling-training-2020-02-23
pan20-author-profiling-training-2020-02-23.zip
sample_data


## Build folders hierarchy to use Keras folders preprocessing function.

In [3]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')
if not os.path.exists('train_dir_es'):
    os.makedirs('train_dir_es')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_es/0'):
    os.makedirs('train_dir_es/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')
if not os.path.exists('train_dir_es/1'):
    os.makedirs('train_dir_es/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')
if not os.path.exists('test_dir_es'):
    os.makedirs('test_dir_es')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_es/0'):
    os.makedirs('test_dir_es/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')
if not os.path.exists('test_dir_es/1'):
    os.makedirs('test_dir_es/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

.config						sample_data
__MACOSX					test_dir_en
pan20-author-profiling-test-2020-02-23		test_dir_es
pan20-author-profiling-test-2020-02-23.zip	train_dir_en
pan20-author-profiling-training-2020-02-23	train_dir_es
pan20-author-profiling-training-2020-02-23.zip


## Set language and directory paths.


In [4]:
# Set en and es train_dir and test_dir paths.
language='es'

truth_file_training_dir_es=training_set_dir+'/'+language+'/'
truth_file_training_path_es = truth_file_training_dir_es+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_es = truth_file_test_dir+'/'+language+'.txt'


language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_path_en = truth_file_test_dir+'/'+language+'.txt'

## Read truth.txt to organize training dataset folders.



In [5]:
# Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

language='es'
# Organize ES folders.
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_es+fNameXml):
      os.rename(truth_file_training_dir_es+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Read truth.txt to organize test dataset folders.

In [6]:
#Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

#Organize EN folders.
language='es'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Function to pre-process source text.

In [7]:
def custom_standardization(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_es_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="es">', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_author_lang_es_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

## Building the dataset.

In [8]:
batch_size=1

# Build the dataset for Spanish.
language='es'

raw_train_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


# Build the dataset for Spanish.
language='en'

raw_train_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


## Convert DSs to Pandas Dataframe

In [9]:
# Convert English dataset.
train_df_en = [] # will contain text and label
for element in raw_train_ds_en:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  train_df_en.append({
      'text':text,
      'label':label
  })
train_df_en = pd.DataFrame(train_df_en)
test_df_en = [] # will contain text and label
for element in raw_test_ds_en:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  test_df_en.append({
      'text':text,
      'label':label
  })
test_df_en = pd.DataFrame(test_df_en)

# Convert Spanish dataset.
train_df_es = [] # will contain text and label
for element in raw_train_ds_es:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  train_df_es.append({
      'text':text,
      'label':label
  })
train_df_es = pd.DataFrame(train_df_es)
test_df_es = [] # will contain text and label
for element in raw_test_ds_es:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  test_df_es.append({
      'text':text,
      'label':label
  })
test_df_es = pd.DataFrame(test_df_es)

## Print some RAW and preprocessed samples (No need to execute)

In [ ]:
for idx, element in enumerate(raw_train_ds_es):
  if idx>1: break
  authorDocument=element[0]
  label=element[1]
  temp = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  print("Not-Preprocessed samples: \n",authorDocument)
  print("Preprocessed samples: \n",temp)

Not-Preprocessed samples: 
 tf.Tensor([b'<author lang="es">\n\t<documents>\n\t\t<document><![CDATA[Movimiento 15M #URL#]]></document>\n\t\t<document><![CDATA[RT #USER#: Alicia , peque\xc3\xb1a... cuanto horror. Nuestro cuerpo roto, al fin de la jornada, lagrimea... cuanto dolor!! #HASHTAG#\xe2\x80\xa6]]></document>\n\t\t<document><![CDATA[He publicado una foto nueva en Facebook #URL#]]></document>\n\t\t<document><![CDATA[El enchufismo en las universidades es tan enorme que cuando los enchufados de grado m\xc3\xa1ximo se jubilan consiguen... #URL#]]></document>\n\t\t<document><![CDATA[He publicado una foto nueva en Facebook #URL#]]></document>\n\t\t<document><![CDATA[RT #USER#: #USER# El que de alguna manera ultraje o menosprecie la Bandera, el Escudo y el Himno de la Rep\xc3\xbablica ser castigado d\xe2\x80\xa6]]></document>\n\t\t<document><![CDATA[#HASHTAG#, espero que tu tesorero sea m\xc3\xa1s leal que #HASHTAG# \nMenos mal que ibais a regenerar la pol\xc3\xadtica en Espa\xc3\xb1a.\

## Model definition.

In [10]:
# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

# English Model Training..
model_en_args = ClassificationArgs(num_train_epochs=10, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
model_en = ClassificationModel("xlnet", 
                                  'xlnet-base-cased', 
                                  args = model_en_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
model_en.train_model(train_df_en[['text', 'label']])

# Spanish Model Training. 
model_es_args = ClassificationArgs(num_train_epochs=10, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
model_es = ClassificationModel("xlnet", 
                                  'xlnet-base-cased', 
                                  args = model_es_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
model_es.train_model(train_df_es[['text', 'label']])

Cuda available?  True


Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

...

Running Epoch 6 of 10:   0%|          | 0/38 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/38 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/38 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/38 [00:00<?, ?it/s]

(380, 0.4722556696910607)

## Evaluate the model on test set

In [11]:
result_en, model_outputs_en, wrong_predictions_en = model_en.eval_model(test_df_en)

result_es, model_outputs_es, wrong_predictions_es = model_es.eval_model(test_df_es)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

## Print results

In [12]:
# Results on english dataset.
print(result_en)
correct_predictions = result_en['tp']+result_en['tn']
print(correct_predictions)
total_predictions = result_en['tp']+result_en['tn']+result_en['fp']+result_en['fn']
print(total_predictions)
accuracy = correct_predictions/total_predictions
print("Accuracy on English test set is:",accuracy)

# Results on spanish dataset.
print(result_es)
correct_predictions = result_es['tp']+result_es['tn']
print(correct_predictions)
total_predictions = result_es['tp']+result_es['tn']+result_es['fp']+result_es['fn']
print(total_predictions)
accuracy = correct_predictions/total_predictions
print("Accuracy on Spanish test set is:",accuracy)

{'mcc': 0.3118926936195077, 'tp': 60, 'tn': 71, 'fp': 29, 'fn': 40, 'auroc': 0.7192500000000001, 'auprc': 0.7000195246900249, 'eval_loss': 2.3646464174985886}
131
200
Accuracy on English test set is: 0.655
{'mcc': 0.3646984043128985, 'tp': 60, 'tn': 76, 'fp': 24, 'fn': 40, 'auroc': 0.7520500000000001, 'auprc': 0.7740410383323506, 'eval_loss': 1.752404807806015}
136
200
Accuracy on Spanish test set is: 0.68
